In [3]:
import cv2 as cv
import numpy as np
import tensorflow as tf

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
import os
import random

In [6]:
from shutil import copyfile

In [9]:
width=0
height=0
tot=0
for dirname, _, filenames in os.walk('input/aii-coffee-arabica/eaii_coffee_arabica/classification'):
    for filename in filenames:
        path=os.path.join('input/aii-coffee-arabica/eaii_coffee_arabica/classification',os.path.join(dirname, filename))
#         print(path)
        image=cv.imread(path)
        width+=image.shape[1]
        height+=image.shape[0]
        tot+=1

In [10]:
print(width/tot,height/tot)

157.7980072463768 163.94067028985506


In [11]:
# GRADED FUNCTION: split_data
def split_data(SOURCE_DIR, TRAINING_DIR, VALIDATION_DIR, SPLIT_SIZE):

    ### START CODE HERE
    files=os.listdir(SOURCE_DIR)
    files=random.sample(files,len(files))
    index=int(SPLIT_SIZE*len(files))
    for i in files[:index]:
        if os.path.getsize(os.path.join(SOURCE_DIR,i))==0:
         print(i, "is zero length, so ignoring.")
        else:
         print(os.path.join(SOURCE_DIR,i))
         copyfile(os.path.join(SOURCE_DIR,i),os.path.join(TRAINING_DIR,i))
        for i in files[index:]:
            if os.path.getsize(os.path.join(SOURCE_DIR,i))==0:
             print(i, "is zero length, so ignoring.")
            else:
             copyfile(os.path.join(SOURCE_DIR,i),os.path.join(VALIDATION_DIR,i))

  ### END CODE HERE


In [12]:

os.mkdir("working/training/")
os.mkdir("working/val/")

os.mkdir("working/training/free_feeder")
os.mkdir("working/training/leaf_rust")
os.mkdir("working/training//leaf_skeletonizer")

os.mkdir("working/val/free_feeder")
os.mkdir("working/val/leaf_rust")
os.mkdir("working/val//leaf_skeletonizer")

In [2]:
split_data("input/aii-coffee-arabica/eaii_coffee_arabica/classification/free_feeder", "/kaggle/working/training/free_feeder", "/kaggle/working/val/free_feeder", 0.85)

In [3]:
split_data("input/aii-coffee-arabica/eaii_coffee_arabica/classification/leaf_rust", "/kaggle/working/training/leaf_rust", "/kaggle/working/val/leaf_rust", 0.85)

In [4]:
split_data("input/aii-coffee-arabica/eaii_coffee_arabica/classification/leaf_skeletonizer", "/kaggle/working/training/leaf_skeletonizer", "/kaggle/working/val/leaf_skeletonizer", 0.85)

In [19]:
def train_val_generators(TRAINING_DIR, VALIDATION_DIR):
 
  train_datagen = ImageDataGenerator(rescale=(1./255),
                                     rotation_range=40,
                                     width_shift_range=.2,
                                     height_shift_range=.2,
                                     shear_range=.1,
                                     zoom_range=0.2,
                                     horizontal_flip=True,
                                     fill_mode="nearest")

  train_generator = train_datagen.flow_from_directory(directory=TRAINING_DIR,
                                                      batch_size=60,
                                                      class_mode="categorical",
                                                      target_size=(150, 150))


  validation_datagen = ImageDataGenerator(rescale=(1./255))

  
  validation_generator = validation_datagen.flow_from_directory(directory=VALIDATION_DIR,
                                                                batch_size=60,
                                                                class_mode="categorical",
                                                                target_size=(150, 150))
  return train_generator, validation_generator


In [20]:
train_generator, validation_generator = train_val_generators("working/training", "working/val")

Found 1843 images belonging to 3 classes.
Found 327 images belonging to 3 classes.


In [21]:
from sklearn.utils import class_weight
import numpy as np

In [22]:
y_train = train_generator.classes

In [23]:
class_weights = class_weight.compute_class_weight('balanced',
                                                  classes=np.unique(y_train),
                                                  y=y_train)
class_weights = dict(enumerate(class_weights))

In [24]:
def create_model():

  model = tf.keras.models.Sequential([ 
      tf.keras.layers.Input(shape=(150,150,3)),
      tf.keras.layers.Conv2D(32,3,activation='relu'),
      tf.keras.layers.MaxPool2D(),
      tf.keras.layers.Conv2D(64,3,activation='relu'),
      tf.keras.layers.MaxPool2D(),
      tf.keras.layers.Conv2D(128,3,activation='relu'),
      tf.keras.layers.MaxPool2D(),
      tf.keras.layers.Conv2D(256,3,activation='relu'),
      tf.keras.layers.MaxPool2D(),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128,activation='relu'),
      tf.keras.layers.Dropout(0.2),
      tf.keras.layers.Dense(3,activation='softmax')
  ])

  

  
  model.compile(optimizer='Rmsprop',
                loss="categorical_crossentropy",
                metrics=['accuracy']) 
  
    

  return model

In [25]:
model=create_model()

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 32)      896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 17, 17, 128)      0

In [27]:
folder_names = [os.path.dirname(f) for f in validation_generator.filenames]

In [5]:
print(folder_names)

In [29]:
folder_index_map = {}
for i, folder_name in enumerate(sorted(set(folder_names))):
    folder_index_map[folder_name] = i

In [30]:
print(folder_index_map)

{'free_feeder': 0, 'leaf_rust': 1, 'leaf_skeletonizer': 2}


In [32]:
model.compile(optimizer="adam",loss="categorical_crossentropy",metrics=["accuracy"])

In [7]:
history = model.fit(train_generator,
                    epochs=50,
                    verbose=1,
                    validation_data=validation_generator,
                    class_weight=class_weights)

In [ ]:
model.evaluate_generator(validation_generator)

In [51]:
model.save("class_new_5.h5")

In [1]:
# print(class_weights)